In [ ]:
# Object Detection

# Importing the libraries
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

# Defining a function that will do the detections
def detect(frame, net, transform): #定義一個檢測函數叫做Detect，Frame是每一個畫面，net是SSD，以及圖像轉換。
    height, width = frame.shape[:2] #提取每一個Frame的寬高
    frame_t = transform(frame)[0] #將Frame進行轉換
    x = torch.from_numpy(frame_t).permute(2, 0, 1) #將Frame轉換為Torch Tensor
    x = Variable(x.unsqueeze(0)) #因為樣本只有一個，所以採用unsqueeze建立一個假維度，Torch不接受單一樣本
    y = net(x) #將神經網路結果輸出y
    detections = y.data #建立一個包含y的Detections Tensor
    scale = torch.Tensor([width, height, width, height])#建立一個符合圖像寬高的Tensor物件
    # detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]
    for i in range(detections.size(1)): #對每一個Class
        j = 0 #初始化循環
        while detections[0, i, j, 0] >= 0.6: #考慮大於0.6的第i類的所有事件j
            pt = (detections[0, i, j, 1:] * scale).numpy() #獲得檢測方框的左上跟右下角座標
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)#繪製方框
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)#加上文字
            j += 1 #增加j到下一個事件
    return frame

# Creating the SSD neural network
net = build_ssd('test') #建立一個Object為SSD
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))
#從另一個神經網路ssd300_mAP_77.43_v2.pth取得權重


# Creating the transformation
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))#用BaseTransform將圖像做必要的轉換，以便做神經網路輸入

# Doing some Object Detection on a video
reader = imageio.get_reader('funny_dog.mp4') #匯入影片
fps = reader.get_meta_data()['fps'] #取得Frame的頻率
writer = imageio.get_writer('output.mp4', fps = fps) #產生影片輸出
for i, frame in enumerate(reader): #進行影片中每一個Frame的迴圈
    frame = ㄏ(frame, net.eval(), transform) #調用前面寫好的detect函數
    writer.append_data(frame) #每一次的Frame都append上去
    print(i) #印出處理好的Frame
writer.close()